In [ ]:
import os
import pandas as pd
from google.cloud import bigquery 
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import SGDClassifier
import numpy as np


In [11]:
file_path = "./.env/service_account.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = file_path
project_id = 'fleet-petal-448410-u6'
dataset_id = "titanic_dataset"
raw_table = 'RAW_train_data'

client = bigquery.Client(project=project_id)
table = raw_table

print(f"Processing: {table}")
query = f"SELECT * FROM {dataset_id}.{table}"
query_job = client.query(query)
results = query_job.result()
df = results.to_dataframe()
print(df.head())
      


Processing: RAW_train_data


/home/yahyaouiabdelkader85/.pyenv/versions/Test-titanic/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


  PassengerId HomePlanet  CryoSleep    Cabin  Destination  Age    VIP  \
0     6145_01     Europa       <NA>  C/231/S  55 Cancri e  NaN  False   
1     0052_01      Earth      False    G/6/S  TRAPPIST-1e  NaN  False   
2     0068_01       Mars      False    E/4/S  TRAPPIST-1e  NaN  False   
3     0202_02     Europa      False    A/2/P  55 Cancri e  NaN  False   
4     0206_01     Europa      False    C/9/S  55 Cancri e  NaN  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0       3478.0       10.0           0.0   105.0  2383.0  Benebah Asolipery   
1          4.0        0.0           2.0  4683.0     0.0   Elaney Hubbarton   
2        793.0        0.0           2.0   253.0     0.0        Cinst Binie   
3          0.0     2433.0           NaN   878.0   443.0      Vegas Embleng   
4          2.0     1720.0          12.0  1125.0   122.0    Nuson Brugashed   

   Transported  
0        False  
1        False  
2        False  
3         True  
4      

# DATA Processing

In [ ]:
#df.isnull().sum()/len(df)

PassengerId     0.000000
HomePlanet      0.023122
CryoSleep       0.024963
Cabin           0.022892
Destination     0.020936
Age             0.020591
VIP             0.023352
RoomService     0.020821
FoodCourt       0.021051
ShoppingMall    0.023927
Spa             0.021051
VRDeck          0.021627
Name            0.023007
Transported     0.000000
dtype: float64

In [15]:
numerical_features = make_column_selector(dtype_include=np.number)
categorial_features = make_column_selector(dtype_exclude=np.number)

In [29]:
numerical_pipeline = make_pipeline(SimpleImputer(strategy= 'median'),StandardScaler())
categorial_pipeline = make_pipeline(SimpleImputer(missing_values=pd.NA,strategy='most_frequent'),OneHotEncoder())


In [ ]:
preprocessor = make_column_transformer((numerical_pipeline,numerical_features),
                                     (categorial_pipeline,categorial_features))
preprocessor.fit(df)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Age', 'RoomService', 'FoodCourt',
                                  'ShoppingMall', 'Spa', 'VRDeck']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(missing_values=<NA>,
                                                                strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f63daee79d0>)])

In [33]:
X, y = df.drop('Transported', axis=1), df['Transported']

In [ ]:
model = make_pipeline(preprocessor, )
preprocessor.fit(X,y)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Age', 'RoomService', 'FoodCourt',
                                  'ShoppingMall', 'Spa', 'VRDeck']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(missing_values=<NA>,
                                                                strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f63daee79d0>)])